In [2]:
import scanpy
import pandas as pd
import numpy as np
import phate
import gspa
import seaborn as sns

In [3]:
data_int = scanpy.read_h5ad('data/int.h5ad')
data_gc = scanpy.read_h5ad('data/GC.h5ad')
hits = data_gc.to_df()
hits.columns = [x.split('-')[0] for x in data_gc.var_names]
hits = hits.groupby(axis=1, level=0).sum()
guide_hit_signal = hits.columns[np.where(hits.values > 0)[1]]
guides = hits.columns

/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [ ]:
for guide in guides:
    data = data_int[guide_hit_signal == guide].to_df()
    # get graph of cells for hit
    phate_op = phate.PHATE(random_state=42, n_jobs=-1, knn=30, verbose=False)
    NEG_phate = phate_op.fit_transform(data)
    G = phate_op.graph.to_pygsp()
    
    # get cell dictionary
    gspa_op = gspa.GSPA(graph=G)
    gspa_op.build_diffusion_operator()
    gspa_op.build_wavelet_dictionary()
    
    # Get signals with norm > 0
    data = data.iloc[:, np.linalg.norm(data, axis=0) > 0]

    # get signal embedding
    signals_projected = gspa_op.embedding.project(data.T, gspa_op.wavelet_dictionary)
    signals_reduced = gspa_op.embedding.svd(signals_projected)
    gene_localization = gspa_op.calculate_localization()
    
    # save
    results = {}
    results['signal_embedding'] = signals_reduced[:, :5]
    results['localization_score'] = gene_localization
    np.savez(f'results/{guide}_GSPA.npz', signal_embedding=results['signal_embedding'],
             localization_score=results['localization_score'], genes=data.columns.values)